##### 유튜브 스크롤링

In [90]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from tqdm import tqdm_notebook

In [91]:
from selenium.webdriver.common.keys import Keys

In [92]:
driver = webdriver.Chrome(ChromeDriverManager().install())

In [93]:
driver.get('https://www.youtube.com/channel/UCSzHok6X5qXEO7cjvVnE62g/videos')

In [94]:
driver.maximize_window()

In [96]:
# 현재 창에 있는 data 불러오기
title = driver.find_elements_by_css_selector('#video-title')
    # 상위태그로 불러오는 방법
    # title = driver.find_elements_by_css_selector
    # ('h3.style-scope ytd-grid-video-renderer > a.yt-simple-endpoint style-scope ytd-grid-video-renderer')
views = driver.find_elements_by_css_selector('#metadata-line > span:nth-child(1)')  # child(1) 같은게 있으면 제대로 data가 불러져 오는지 확인해야됨
title_list = []
views_list = []
for i in title:
    title_list.append(i.text)
for i in views:
    views_list.append(i.text)

title_list, views_list, len(title_list), len(views_list)

(['[LOL] 프레이 아무무 : 어이! 어이! 믿고있었다구!!!',
  '[LOL] 프레이 닐라 : 오늘만큼은 망나니 원딜이 되겠다.',
  '[LOL] 영감 미안하오.. 멸망전땐 안하려했지만 너무 꿀잼이오ㅋㅋ',
  '[LOL] 프레이의 애쉬는 멸망전에서도 다르다',
  '[LOL] 멸망전 서폿 프레이에서 이상호가 된 나',
  "[LOL] 멸망전 서폿 '답답하면 니들이 뛰던지'에서 뛰는놈입니다",
  '[LOL] 멸망전 서폿 신입사원 프레이 인사박습니다!',
  '[LOL] 프레이 진 : 최고의 조합이면서 최악의 서폿녀석 내CS...',
  '[LOL] 프레이 아무무 : 어디 가는거야? 우웅..나랑 놀자.. 나랑 놀자고!!!',
  '[LOL] 프레이 아펠리오스 : 천상계에서 의문의 케인장인에게 캐리받기',
  '[LOL] 프레이 세나 : 이게 세나고!! 이게 탐켄치고!! 이게 세탐이다!!!',
  '[LOL] 프레이 카이사 : 마형 힘의 차이가 느껴져? 버프된 카이사야',
  '[LOL] 프레이 케이틀린 : 나 버프먹은 케틀이야 3대2도 가능해 드루와 드루와',
  '[LOL] 프레이 칼리스타 : 망할 저격수와 디코하면서 콩자반 맛보기',
  '[LOL] 프레이 닐라 : 아 왕호야 진짜 그러지마라 하지말라했다!!',
  '[LOL] 프레이 시비르 : 인간이 5명 모이면 반드시 1명은 못하는 사람이 있다',
  '[LOL] 프레이 닐라 : 이게 케리아..? 구마유시 체험하기',
  '[LOL] 프레이 블리츠크랭크 : 어? 협곡에서 강제 팬미팅하기',
  '[LOL] 프레이 닐라 : 그면상? 이젠 내면상이다 ㅋㅋㅋㅋ',
  '[LOL] 프레이 케이틀린 : 나를 죽이지 못하는 고통은 나를 더욱 강하게 만든다',
  '[LOL] 프레이 콩자반 : 죄송합니다. 보시다시피 저희 원딜도 반성하고 있습니다',
  '[LOL] 프레이 칼리스타 : 칼리만 잡으면 더 강해지는 남자',
  '[LOL] 프레이 바루스 : 바텀메타? 바텀론? 일단 정글뽑기부터 잘해야한다',
 

##### 스크롤링

In [97]:
from bs4 import BeautifulSoup as bs
import requests as req

In [98]:
body = driver.find_element_by_css_selector('body')

In [99]:
# 한번 실행할 때마다 한번씩 스크롤링 되기 때문에 while문 사용하여 끝까지 내리고 data를 받아오기
# 1. 스크롤 높이를 이용해서 break하는 방법 -> 얼마나 내려갈지 모르기 때문에 높이를 정의해주기 힘듬
# 2. soup으로 객체화를 이용한 방법 : 스크롤한 page를 soup을 통해 객체화 
#   -> 일정 횟수만큼 스크롤링을 한 후(end키 사용) 객체화한 부분과 이전 객체화한 부분이 같은지 비교
#   -> 스크롤링 전/후 객체화가 같으면 스크롤 끝냄(같으면 내용이 같다고 생각하여 끝냄)
while tqdm_notebook(True):
    last = bs(driver.page_source, 'html.parser')    # driver.page_source : page 가져옴
    for i in range(10):
        body.send_keys(Keys.END)    # 스크롤 한번 내리기
    new = bs(driver.page_source,'html.parser')
    if (last == new):
        break
    else:
        pass

In [100]:
title = driver.find_elements_by_css_selector('#video-title')
views = driver.find_elements_by_css_selector('#metadata-line > span:nth-child(1)')

In [135]:
title_list = []
views_list = []
for i in tqdm_notebook(title):
    title_list.append(i.text)
for i in tqdm_notebook(views):
    string = i.text
    string = string.replace("조회수 ","")
    string = string.replace("회","")
    if ('.' in string):
        if ('천' in string):
            string = string.replace("천","")
            string = float(string)  # 소수점은 float사용 해줘야 됨(int는 안됨)
            string = string*1000
            views_list.append(string)
        elif ('만' in string):
            string = string.replace("만","")
            string = float(string)
            string = string*10000
            views_list.append(string)
        else:
            pass
    else:
        string = string.replace("천","000")
        string = string.replace("만","0000")
        string = float(string)  # 나중에 정렬해주기 위해서 형변환(String과 float,int 비교가 안되기 때문)
        views_list.append(string)

C:\Users\user\AppData\Local\Temp\ipykernel_13232\1583781517.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(title):


  0%|          | 0/1368 [00:00<?, ?it/s]

C:\Users\user\AppData\Local\Temp\ipykernel_13232\1583781517.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(views):


  0%|          | 0/1368 [00:00<?, ?it/s]

In [136]:
len(title_list), len(views_list)

(1368, 1368)

In [137]:
youtube_data = {'제목':title_list,'조회수':views_list}
pray_youtube = pd.DataFrame(youtube_data)
pray_youtube

,제목,조회수
0,[LOL] 프레이 아무무 : 어이! 어이! 믿고있었다구!!!,43000.0
1,[LOL] 프레이 닐라 : 오늘만큼은 망나니 원딜이 되겠다.,79000.0
2,[LOL] 영감 미안하오.. 멸망전땐 안하려했지만 너무 꿀잼이오ㅋㅋ,140000.0
3,[LOL] 프레이의 애쉬는 멸망전에서도 다르다,130000.0
4,[LOL] 멸망전 서폿 프레이에서 이상호가 된 나,80000.0
...,...,...
1363,[LOL] 프레이 탑신짜오 배달왔더니 응원만하네_170523,53000.0
1364,[LOL] 프레이 자크서폿 핑크젤리 싸대기_170525,33000.0
1365,[LOL] 프레이와 스맵의 대벌레듀오 PAX트페원딜&정글자크_170525,270000.0
1366,[LOL] 프레이 솔랭에서 만난 고릴라 어제의 동료가 오늘의 적_170525,68000.0


In [138]:
# 조회수는 Stirng type이기 때문에 '회'를 제거하고 만을 숫자로 치환한 후 sortvalue로 정렬
df = pd.DataFrame(youtube_data)
df = df.sort_values(by=['조회수'], ascending=False)
df = df.reset_index(drop=True)    # 인덱스 새로 지정
df

,제목,조회수
0,[LOL] 프레이vs중급봇 1대5 스킬 안쓰고 평타로만 깨기,2200000.0
1,[주간] 페이커가 챌린저로 시즌 마감을 안하는 이유,1410000.0
2,[LOL] 프레이의 미친무빙 탑 이속 진 : 어딜노리는겁니까 그건 잔상입니다_170531,1360000.0
3,[LOL] [#1] 프레이 이즈리얼 : 2달 만에 롤 했더니 챌린저였던 내가 브론즈...,1340000.0
4,[LOL] 프레이 도벽 룬 이즈리얼 : 프레이와 스맵 대벌레듀오! 웬만해선 그들을 ...,1290000.0
...,...,...
1363,[FULL TALK] 프레이 LZ VS AFs 경기 후 토크방송(PraY's Tal...,27000.0
1364,[PUBG] 프레이의 배틀그라운드 : 프레이와 라스칼 랜덤스쿼드 뚝배기 킬러!_17...,27000.0
1365,[FULL TALK] [2] 프레이 LZ VS EEW 경기 후 토크방송(PraY's...,24000.0
1366,[FULL TALK] 프레이 LZ VS MVP 경기 후 토크방송(PraY's Tal...,21000.0
